#### Establish baseline

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

from keras import backend as K
num_gpus = len(K.tensorflow_backend._get_available_gpus())

from keras import layers
from keras import models

Using TensorFlow backend.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

PATH = '/root/data/sar_cars/'
df = pd.read_csv(os.path.join(PATH, 'labels_balanced.csv'))
classes = list(df.car.unique())
print(df.car.value_counts())
df_train, df_test = train_test_split(df, test_size=0.2)
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)
df_train.head(3)

Chevrolet_Impala_LT        6925
Pontiac_Torrent            6924
Toyota_Highlander          6924
Mitsubishi_Galant_ES       6924
Chevrolet_HHR_LT           6924
Chrysler_Town_&_Country    6924
Name: car, dtype: int64


,id,car
0,PH_fcara2_0222_88deg_az,Chevrolet_Impala_LT
1,PH_fcarb2_0221_163deg_az,Mitsubishi_Galant_ES
2,PH_fcarb2_0222_22deg_az,Mitsubishi_Galant_ES


In [4]:
import numpy as np
import PIL

def batch_generator(df):
    while True:
        for start in range(0, df.shape[0]-1, batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, df.shape[0]-1)
            ids_batch = list(range(start,end))
            for i in ids_batch:           
                img = np.array(PIL.Image.open(os.path.join(PATH, 'train', df.iloc[i].id + '.png')))/255
                labelname=df.iloc[i]['car']
                labelnum=classes.index(labelname)
                x_batch.append(img)
                y_batch.append(labelnum)
            x_batch_np = np.array(x_batch, dtype=np.float32)
            y_batch_np = np.zeros((len(y_batch), len(classes)))     
            y_batch_np[np.arange(len(y_batch)),y_batch] = 1
            yield x_batch_np, y_batch_np

Prebuilt Model

In [7]:
assert model.trainable

[<tf.Variable 'stem_conv1/kernel:0' shape=(3, 3, 3, 96) dtype=float32_ref>,
 <tf.Variable 'stem_bn1/gamma:0' shape=(96,) dtype=float32_ref>,
 <tf.Variable 'stem_bn1/beta:0' shape=(96,) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/reduction_conv_1_stem_1/kernel:0' shape=(1, 1, 96, 42) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/reduction_bn_1_stem_1/gamma:0' shape=(42,) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/reduction_bn_1_stem_1/beta:0' shape=(42,) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/block_1/separable_conv_block_reduction_left1_stem_1/separable_conv_1_reduction_left1_stem_1/depthwise_kernel:0' shape=(5, 5, 42, 1) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/block_1/separable_conv_block_reduction_left1_stem_1/separable_conv_1_reduction_left1_stem_1/pointwise_kernel:0' shape=(1, 1, 42, 42) dtype=float32_ref>,
 <tf.Variable 'reduction_A_block_stem_1/block_1/separable_conv_block_reduction_right1_stem

In [ ]:
from keras.applications import NASNetLarge, InceptionResNetV2
from keras.utils import multi_gpu_model
from keras import optimizers

batch_size_per_gpu = 16
batch_size = batch_size_per_gpu*num_gpus
epochs = 30

#model = NASNetLarge(weights=None,include_top=True, input_shape=(160,160,3),classes=6)
model = InceptionResNetV2(weights=None,include_top=True, input_shape=(160,160,3),classes=6)
#model.summary()
assert model.trainable

parallel_model = multi_gpu_model(model,gpus=num_gpus)

parallel_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = parallel_model.fit_generator(generator=batch_generator(df_train), 
               steps_per_epoch= int(np.ceil(float(df_train.shape[0]-1) / float(batch_size))),
                epochs=epochs, verbose=1, 
                validation_data=batch_generator(df_test), 
                validation_steps=int(np.ceil(float(df_test.shape[0]-1) / float(batch_size))))

# nasnat clearly overfitting